## PCRデータの陰性陽性判定プログラムを作ろう

### 課題 
Aを基本課題（全員必須）、Bを自由課題（希望者のみ）とする。
Aで設定したルールの背景と可視化結果を分かりやすく説明してください。
B の対応をされた方は、Bを中心に説明下さい。
- A: PCRのシグナルデータを可視化し、PCRの原理を踏まえて陰性/陽性を決めるルールを決定してプログラムで表現する
- B: シグナルと正解ラベルのデータから分類器を作成し、精度評価を行う

### データセット
- scratch/shared/data/signals.csv
- scratch/shared/data/labels.csv
    - ラベルのデータは演習のための人工的に作成したものになっています

### 課題Aのマイルストーン
シグナルデータだけを扱います。
1. Numpyが使えるようになる
2. matplotlibを利用してシグナルデータを可視化する
3. PCRの原理を踏まえ、陰性/陽性を区別するルールを自分で設定する
    - 例）〇〇サイクル目でシグナルが△△以上のものを陽性としよう
4. シグナルデータにルールを適用し、陰性/陽性がわかるように可視化する

### 課題Bのマイルストーン
シグナルと正解ラベルデータを扱います。
1. シグナルと正解ラベルの対応関係の特徴をつかみ、どのように処理すると陰性陽性が判定できるか考える
    - 最大値? 特定のサイクルの値で判定? 機械学習?
2. PCRのシグナルデータを入力すると、陰性/陽性を出力するプログラムを作成する
3. 予測結果を作成する
4. 正解と予測結果を照らし合わせ、精度評価を行う
    - 正解率、混同行列、感度、特異度、ROC曲線、...
  
### 課題発表について
- 5/24 (金) 13:00-15:00
- 一人あたり 発表：10分、質疑応答：5分程度
- 発表内容：簡易な自己紹介、課題、AI研修の振り返り

In [ ]:
# パッケージのインストール・読み込み
!pip install numpy pandas scikit-learn matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Numpyの基本
本題に入る前に、`numpy`の基本について触れます。このパッケージの主役である`numpy.ndarray`はPython標準の`list`を入れ子にしたようなオブジェクトです。
<img src="https://qph.fs.quoracdn.net/main-qimg-30be20ab9458b5865b526d287b4fef9a" width="500" >  
今回は２次元のデータを扱います。配列の操作の基本に触れます。

In [ ]:
# 例となる配列を生成。np.arange()関数はPython標準のrange()関数と同じようなものです
example = np.arange(15).reshape(3, 5)
example

`ndarray`で気にかけておくべきことは次の3つです。
- 配列の次元 (`ndarray.ndim`)
- 配列の形 (`ndarray.shape`)
- 配列のデータ型 (`ndarray.dtype`): 1つの`ndarray`オブジェクトには1種類のデータ型（int, floatなど）しか入りません

In [ ]:
example.ndim, example.shape, example.dtype

In [ ]:
# 配列から抽出する操作
print(example[1, 3])  # 2次元の場合、[y軸のインデックス, x軸のインデックス] というイメージ。インデックスはゼロ始まりです

# 行の抽出
print(example[1])

# 列の抽出
print(example[:, 2])  # コロン: は「その軸について全て」

# 部分的に抽出
print(example[0:2, 2:4])

In [ ]:
# numpyの関数にはaxis 軸を指定することができます
print(np.mean(example))  # 全体の平均
print(np.mean(example, axis=0))  # axis=0 y軸方向に平均 shape(3, 5) -> (_, 5)
print(np.mean(example, axis=1))  # axis=1 x軸方向に平均 shape(3, 5) -> (3, _)

# その他よく使うもの
print(np.sum(example, axis=0))  # 合計
print(np.min(example, axis=0))  # 最小値
print(np.max(example, axis=0))  # 最大値


In [ ]:
# 配列と数値（スカラー）の計算は、それぞれの要素の計算となります
print(example + 100)

# 同じ形の配列どうしの計算も要素ごとです
print(example * example)

# 発展：異なる形の配列の計算は、ブロードキャストと呼ばれるルールによって形状を同じにすることができれば計算可能です 
# 詳しくは https://note.nkmk.me/python-numpy-broadcasting/
print(example + [0, 10, 100, 1000, 10000])  # ブロードキャスト可能 (3,5), (5,) -> (3,5), (1,5) -> (3,5), (3,5)
print(example + [0, 1])  # ブロードキャスト不可 (3,5), (2,) -> (3,5), (1,2) -> (3,5), (3,2)  不一致

## 課題A チュートリアル

In [ ]:
# CSVファイルを配列に読み込み
signals = np.loadtxt('scratch/shared/data/signals.csv', delimiter=',')
signals

In [ ]:
#データの形(shape)、型(dtype)を調べます
print("形：", signals.shape)
print("型：", signals.dtype)

In [ ]:
# このデータは縦軸にサンプル、横軸に各サイクルにおけるシグナル強度が入っています
# 1つのサンプルの値を見てみましょう
sample_index = 100

print(signals[sample_index])
plt.plot(signals[sample_index])
plt.show()

In [ ]:
# データ全体を可視化するとどうなるでしょうか? （...のところに書き込んでみてください。...は消去してください。）
for signal in signals:
    ...

plt.show()

In [ ]:
# オリジナルの判定ルールを考えます。例として、シグナルの最大値が3より大きいものを陽性とします
maximum = signals.max(axis=1)  # 各サンプルのシグナル最大値を計算
maximum_bool = maximum > 3  # 最大値が3より大きいか判定。結果はTrueかFalseで表現されています。これを使えば陰性・陽性の可視化ができそうです
maximum_bool

[matplotlib参考](https://utokyo-ipp.github.io/appendix/5-matplotlib.html#:~:text=plot()%20%E9%96%A2%E6%95%B0%E3%81%A7%E3%81%AF%E3%82%B0%E3%83%A9%E3%83%95%E3%81%AE%E7%B7%9A%E3%81%AE%E8%89%B2%E3%80%81%E5%BD%A2%E7%8A%B6%E3%80%81%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9D%E3%82%A4%E3%83%B3%E3%83%88%E3%81%AE%E3%83%9E%E3%83%BC%E3%82%AB%E3%81%AE%E7%A8%AE%E9%A1%9E%E3%82%92%E3%80%81%E3%81%9D%E3%82%8C%E3%81%9E%E3%82%8C%E4%BB%A5%E4%B8%8B%E3%81%AE%E3%82%88%E3%81%86%E3%81%AB%20linestyle%2C%20color%2C%20marker%20%E5%BC%95%E6%95%B0%E3%81%A7%E6%8C%87%E5%AE%9A%E3%81%97%E3%81%A6%E5%A4%89%E6%9B%B4%E3%81%99%E3%82%8B%E3%81%93%E3%81%A8%E3%81%8C%E3%81%A7%E3%81%8D%E3%81%BE%E3%81%99%E3%80%82%E3%81%9D%E3%82%8C%E3%81%9E%E3%82%8C%E3%81%AE%E5%BC%95%E6%95%B0%E3%81%A7%E6%8C%87%E5%AE%9A%E5%8F%AF%E8%83%BD%E3%81%AA%E5%80%A4%E3%81%AF%E4%BB%A5%E4%B8%8B%E3%82%92%E5%8F%82%E7%85%A7%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82)

In [ ]:
# plt.plotを着色するには？
plt.plot([1, 3, 5], color='red')
plt.plot([2, 4, 6], color='blue')
plt.show()

In [ ]:
# どうすればsignal[sample_index]の最大値が3より大きかったら(true) 赤、そうでなかったら(false) 青で折れ線グラフを描写できるでしょうか？
# （...のところにかきこんでみてください。...は消去してください。）
print(signals[sample_index])
print("最大値が3より大きいかどうか：", maximum_bool[sample_index])

if ... :
    plt.plot(...)
else:
    ...

## 課題B チュートリアル

In [ ]:
# CSVファイルを配列に読み込み
signals = np.loadtxt('scratch/shared/data/signals.csv', delimiter=',')
signals.shape, signals.dtype, signals

In [ ]:
# このデータは縦軸にサンプル、横軸に各サイクルにおけるシグナル強度が入っています
# 1つのサンプルの値を見てみましょう
sample_index = 100

print(signals[sample_index])
plt.plot(signals[sample_index])
plt.show()

In [ ]:
# 正解ラベルのデータも確認します
labels = np.loadtxt('scratch/shared/data/labels.csv', delimiter=',', dtype=int)
labels.shape, labels.dtype, labels

In [ ]:
# ラベルデータには各サンプル（シグナルと同じ順）の陰性[0] 陽性[1]の値が入っています
# 先ほど確認したサンプルはどちらでしょうか
labels[sample_index]

In [ ]:
# シグナルデータ全体のグラフをラベル別に色分けしてみます
n = len(signals)
for i in range(n):  # 各サンプルのインデックスについて、
    signal = signals[i]
    label = labels[i]
    if label == 0:  # サンプルが陰性ならグレー、陽性なら赤で着色
        color = 'gray'
    else:
        color = 'red'
    plt.plot(signal, color=color, alpha=0.2)  # alpha=0.2は透明度を指定（小さいほど透明）

plt.show()    

In [ ]:
# シグナルの最大値を使って判定するのはどうでしょうか？適当な閾値より上かどうかで判定するプログラムを作成します
threshold = 3
signal_max = signals.max(axis=1)  # 各サンプルの最大のシグナル
print(signal_max.shape)

predictions = signal_max > threshold  # 閾値と比較
print(predictions.dtype, predictions[0])  # 比較結果はbool値（TrueかFalse）
predictions = predictions.astype(int)  # 数値に変換 True -> 1, False -> 0
print(predictions.dtype, predictions[0])
predictions

In [ ]:
# 予測結果と正解ラベルを照らし合わせてみます。Trueのサンプルは正しい予測ができていたサンプルです
is_correct = predictions == labels
is_correct

In [ ]:
# 精度評価
# 正解率を算出します
num_correct = np.count_nonzero(is_correct)
num_all = len(is_correct)
accuracy = num_correct / num_all
accuracy

In [ ]:
# 混同行列を作成します。混同行列を利用することで他の指標を計算することができます
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(labels, predictions)
matrix

<img src="https://aiacademy.jp/assets/images_test/34_16bf47ea3d3.png" width="500">